## Capstone
### Attempts to clean up the Moodey Data
- keeping all forecasting data

In [2]:
from openpyxl import load_workbook

# Load the Excel file
file_path = 'Historical_Moodey.xlsm'
output_file_path = 'Historical_Moodey_Values_Only.xlsx'

# Load the workbook
wb = load_workbook(file_path, data_only=True)

# Save a new file with only the calculated values
wb.save(output_file_path)

print(f"Saved values-only workbook to {output_file_path}")

Saved values-only workbook to Historical_Moodey_Values_Only.xlsx


## Attempting to make it more vertical

In [20]:
import pandas as pd

# File path to the Excel workbook
file_path = "Historical_Moodey_Values_Only.xlsx"

# Actual sheet names
sheet_names = [
    "Gross Metro Product", "Total Households", "Total Population", "Total Real Personal Income",
    "Disposable Real Pers Income", "Persons Per Household", "Res SF Permits", "Res MF Permits",
    "Total Res Permits", "Existing SF Home Sales", "Employment Tot Non Agric", "Total Employed",
    "CPI Index", "Fla - Gross State Product", "Fla - Nbr of Households", "Fla - Total Population",
    "Fla -Total Real Personal Income", "Fla - Dis Real Personal Income", "Fla Total RealPers Inc per Hsld",
    "Fla - Emplymt Tot NonAgricultur", "Fla - Tot Resid Permits", "Fla - Persons Per Hsld"
]

# Create a dictionary to store cleaned DataFrames for each sheet
cleaned_sheets = {}

# Process each sheet
for sheet_name in sheet_names:
    try:
        print(f"\nProcessing sheet: {sheet_name}")

        # Load the sheet
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

        # Drop fully empty rows and columns
        df = df.dropna(how="all", axis=0).dropna(how="all", axis=1).reset_index(drop=True)

        # Ensure there are enough rows to drop the first one
        if len(df) < 2:
            print(f"Sheet {sheet_name} appears to have insufficient rows after cleaning.")
            continue

        # Drop the first row
        df = df.iloc[1:].reset_index(drop=True)

        # Take the new row 1 as potential headers
        new_headers = df.iloc[0]
        df = df[1:].reset_index(drop=True)
        df.columns = new_headers

        # Ensure the first two columns remain intact
        id_vars = df.columns[:2]

        # Check if the remaining columns have consistent datetime values
        def is_datetime(value):
            try:
                pd.to_datetime(value, format='%b-%y', errors='raise')
                return True
            except ValueError:
                return False

        # Retain only valid datetime columns
        valid_columns = [col for col in df.columns[2:] if is_datetime(col)]
        df = df[id_vars.tolist() + valid_columns]

        # Store the cleaned DataFrame
        cleaned_sheets[sheet_name] = df

        print(f"Sheet {sheet_name} cleaned successfully. Retained columns: {len(valid_columns) + 2}")
    except Exception as e:
        print(f"Error processing sheet {sheet_name}: {e}")

# Optionally save the cleaned sheets to a new Excel file with separate tabs
with pd.ExcelWriter("cleaned_data_datetime.xlsx") as writer:
    for sheet_name, cleaned_df in cleaned_sheets.items():
        cleaned_df.to_excel(writer, sheet_name=sheet_name, index=False)

print("\nAll sheets cleaned and saved to cleaned_data_datetime.xlsx.")



Processing sheet: Gross Metro Product
Sheet Gross Metro Product cleaned successfully. Retained columns: 85

Processing sheet: Total Households
Sheet Total Households cleaned successfully. Retained columns: 84

Processing sheet: Total Population
Sheet Total Population cleaned successfully. Retained columns: 84

Processing sheet: Total Real Personal Income
Sheet Total Real Personal Income cleaned successfully. Retained columns: 33

Processing sheet: Disposable Real Pers Income
Sheet Disposable Real Pers Income cleaned successfully. Retained columns: 33

Processing sheet: Persons Per Household
Sheet Persons Per Household cleaned successfully. Retained columns: 84

Processing sheet: Res SF Permits
Sheet Res SF Permits cleaned successfully. Retained columns: 87

Processing sheet: Res MF Permits
Sheet Res MF Permits cleaned successfully. Retained columns: 88

Processing sheet: Total Res Permits
Sheet Total Res Permits cleaned successfully. Retained columns: 87

Processing sheet: Existing SF

Converting to vertical format

## MANUALLY WENT IN AND DELETED DUPLICATE YEAR COLUMN FROM A FEW SHEETS - wasn't recognizing the duplicates when trying to code the solution

original = cleaned_data_datetime

updated after deleting duplicate columns = cleaned_data_datetime_fixed

In [24]:
import pandas as pd

# Load the cleaned file from the previous step
file_path = "cleaned_data_datetime_fixed.xlsx"

# Load all sheets
cleaned_data = pd.ExcelFile(file_path)

# Create a dictionary to store reformatted DataFrames
vertical_sheets = {}

# Process each cleaned sheet
for sheet_name in cleaned_data.sheet_names:
    try:
        print(f"\nProcessing sheet: {sheet_name}")

        # Load the sheet
        df = cleaned_data.parse(sheet_name)

        # Ensure the first two columns are retained as identifiers
        id_vars = df.columns[:2]

        # Handle duplicate 'Year' columns
        if id_vars[0] == 'Year' and id_vars[1] == 'Year':
            # Drop the second 'Year' column
            df = df.drop(df.columns[1], axis=1)
            id_vars = df.columns[:2]  # Reassign id_vars after dropping

        # Melt the rest of the columns into long format
        melted_df = pd.melt(df, id_vars=id_vars, var_name="Date", value_name=sheet_name)

        # Store the reformatted DataFrame
        vertical_sheets[sheet_name] = melted_df

        print(f"Sheet {sheet_name} converted to vertical format successfully.")
    except Exception as e:
        print(f"Error processing sheet {sheet_name}: {e}")

# Optionally save all sheets into a new Excel file with separate tabs
with pd.ExcelWriter("vertical_data_sheetname_handling_duplicates.xlsx") as writer:
    for sheet_name, vertical_df in vertical_sheets.items():
        vertical_df.to_excel(writer, sheet_name=sheet_name, index=False)

print("\nAll sheets reformatted to vertical format with sheet name as column saved")



Processing sheet: Gross Metro Product
Sheet Gross Metro Product converted to vertical format successfully.

Processing sheet: Total Households
Sheet Total Households converted to vertical format successfully.

Processing sheet: Total Population
Sheet Total Population converted to vertical format successfully.

Processing sheet: Total Real Personal Income
Sheet Total Real Personal Income converted to vertical format successfully.

Processing sheet: Disposable Real Pers Income
Sheet Disposable Real Pers Income converted to vertical format successfully.

Processing sheet: Persons Per Household
Sheet Persons Per Household converted to vertical format successfully.

Processing sheet: Res SF Permits
Sheet Res SF Permits converted to vertical format successfully.

Processing sheet: Res MF Permits
Sheet Res MF Permits converted to vertical format successfully.

Processing sheet: Total Res Permits
Sheet Total Res Permits converted to vertical format successfully.

Processing sheet: Employment 

Cleaning up non-year columns e.g. '5 YR CAGR'

In [26]:
import pandas as pd

# Load the cleaned vertical file
file_path = "vertical_data_sheetname_handling_duplicates.xlsx"

# Load all sheets
vertical_data = pd.ExcelFile(file_path)

# Create a dictionary to store filtered DataFrames
filtered_sheets = {}

# Process each sheet
for sheet_name in vertical_data.sheet_names:
    try:
        print(f"\nProcessing sheet: {sheet_name}")

        # Load the sheet
        df = vertical_data.parse(sheet_name)

        # Filter rows where the first column is numeric (valid years)
        filtered_df = df[pd.to_numeric(df.iloc[:, 0], errors='coerce').notna()]

        # Store the filtered DataFrame
        filtered_sheets[sheet_name] = filtered_df

        print(f"Filtered sheet {sheet_name} successfully.")
    except Exception as e:
        print(f"Error processing sheet {sheet_name}: {e}")

# Optionally save all filtered sheets into a new Excel file with separate tabs
with pd.ExcelWriter("filtered_vertical_data.xlsx") as writer:
    for sheet_name, filtered_df in filtered_sheets.items():
        filtered_df.to_excel(writer, sheet_name=sheet_name, index=False)

print("\nAll sheets filtered and saved to filtered_vertical_data.xlsx.")



Processing sheet: Gross Metro Product
Filtered sheet Gross Metro Product successfully.

Processing sheet: Total Households
Filtered sheet Total Households successfully.

Processing sheet: Total Population
Filtered sheet Total Population successfully.

Processing sheet: Total Real Personal Income
Filtered sheet Total Real Personal Income successfully.

Processing sheet: Disposable Real Pers Income
Filtered sheet Disposable Real Pers Income successfully.

Processing sheet: Persons Per Household
Filtered sheet Persons Per Household successfully.

Processing sheet: Res SF Permits
Filtered sheet Res SF Permits successfully.

Processing sheet: Res MF Permits
Filtered sheet Res MF Permits successfully.

Processing sheet: Total Res Permits
Filtered sheet Total Res Permits successfully.

Processing sheet: Employment Tot Non Agric
Filtered sheet Employment Tot Non Agric successfully.

Processing sheet: Total Employed
Filtered sheet Total Employed successfully.

Processing sheet: CPI Index
Filte

Attempting to Merge the sheets together into one

In [29]:
import pandas as pd

# Load the Excel file
file_path = "filtered_vertical_data.xlsx"
excel_data = pd.ExcelFile(file_path)

# Create a dictionary to store cleaned DataFrames
processed_sheets = {}

# Process each sheet
for sheet_name in excel_data.sheet_names:
    try:
        print(f"\nProcessing sheet: {sheet_name}")

        # Load the sheet
        df = excel_data.parse(sheet_name)

        # Ensure the necessary columns exist
        if "Date" not in df.columns:
            print(f"No 'Date' column found in sheet: {sheet_name}")
            continue

        # Convert the 'Date' column to datetime format, dropping invalid dates
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df = df.dropna(subset=["Date"]).reset_index(drop=True)

        # Convert 'Date' to month-year format
        df["Date"] = df["Date"].dt.to_period("M").dt.to_timestamp()

        # Keep only relevant columns
        id_vars = ["Year", "Quarter/Year", "Date"]
        data_columns = [col for col in df.columns if col not in id_vars]
        df = df[id_vars + data_columns]

        # Store the processed DataFrame
        processed_sheets[sheet_name] = df
    except Exception as e:
        print(f"Error processing sheet {sheet_name}: {e}")

# Merge all sheets together on the first three columns
merged_data = None

for sheet_name, df in processed_sheets.items():
    if merged_data is None:
        # Initialize with the first sheet
        merged_data = df
    else:
        # Merge subsequent sheets
        merged_data = pd.merge(merged_data, df, on=["Year", "Quarter/Year", "Date"], how="outer")

# Remove rows where all values from the 4th column onward are NaN
merged_data = merged_data.dropna(how="all", subset=merged_data.columns[3:])

# Save the merged result to a new Excel file
output_file = "combined_Moodey_data_merged.xlsx"
merged_data.to_excel(output_file, index=False)

print(f"\nAll sheets processed, merged, and saved to {output_file}.")



Processing sheet: Gross Metro Product

Processing sheet: Total Households

Processing sheet: Total Population

Processing sheet: Total Real Personal Income

Processing sheet: Disposable Real Pers Income

Processing sheet: Persons Per Household

Processing sheet: Res SF Permits

Processing sheet: Res MF Permits

Processing sheet: Total Res Permits

Processing sheet: Employment Tot Non Agric

Processing sheet: Total Employed

Processing sheet: CPI Index

Processing sheet: Fla - Gross State Product

Processing sheet: Fla - Nbr of Households

Processing sheet: Fla - Total Population

Processing sheet: Fla -Total Real Personal Income

Processing sheet: Fla - Dis Real Personal Income

Processing sheet: Fla Total RealPers Inc per Hsld

Processing sheet: Fla - Emplymt Tot NonAgricultur

Processing sheet: Fla - Tot Resid Permits

Processing sheet: Fla - Persons Per Hsld

All sheets processed, merged, and saved to combined_Moodey_data_merged.xlsx.
